In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [10]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm

from lightgbm import LGBMClassifier

import warnings
warnings.filterwarnings(action='ignore')
from sklearn.metrics import *

In [3]:
import os
os.chdir('/content/drive/MyDrive/데이콘/exercise_motion_classification/data/')

train=pd.read_csv('train_features.csv')
train_labels=pd.read_csv('train_labels.csv')
test=pd.read_csv('test_features.csv')
submission=pd.read_csv('sample_submission.csv')

In [4]:
train.head()

,id,time,acc_x,acc_y,acc_z,gy_x,gy_y,gy_z
0,0,0,1.206087,-0.179371,-0.148447,-0.591608,-30.549010,-31.676112
1,0,1,1.287696,-0.198974,-0.182444,0.303100,-39.139103,-24.927216
2,0,2,1.304609,-0.195114,-0.253382,-3.617278,-44.122565,-25.019629
3,0,3,1.293095,-0.230366,-0.215210,2.712986,-53.597843,-27.454013
4,0,4,1.300887,-0.187757,-0.222523,4.286707,-57.906561,-27.961234


- train데이터를 id개수별로 맞춰주기 위하여 mean, median, max, min등의 값을 구해서 변수로 추가해보자. 

- 이 방법은 https://dacon.io/competitions/official/235689/codeshare/2306?page=1&dtype=recent&ptype=pub 의 SDSTony님의 코드공유 노트북을 참고하여 진행했습니다. 

In [52]:
features = ['id','acc_x','acc_y','acc_z','gy_x','gy_y','gy_z']
X_train = train[features].groupby('id').agg(['max','min','mean','median'])
X_test = test[features].groupby('id').agg(['max','min','mean','median'])

In [49]:
y_train = train_labels['label']

In [76]:
len(y_train.unique())

61

In [50]:
X_train.shape,y_train.shape

((3125, 24), (3125,))

In [53]:
X_train.reset_index(inplace=True)
X_train.drop(['id'],axis=1,inplace=True)
X_train

acc_x                      ...        gy_z                     
           max       min      mean  ...         min      mean     median
0     1.344268  0.591940  0.931329  ...  -79.930029  1.182107   1.913286
1     1.234020 -2.156208 -0.766580  ... -270.980823  1.393294  -0.655819
2     1.219836 -1.142847  0.039836  ...  -44.192071  3.053291   3.568888
3    -0.622250 -1.417751 -0.887702  ...  -85.600536 -5.869898  -3.853078
4     0.599720 -2.429109 -0.659018  ... -270.581913  4.453382  -0.750283
...        ...       ...       ...  ...         ...       ...        ...
3120  0.390798 -1.624711 -0.300454  ...  -79.392292 -0.054026  -2.190464
3121 -0.446650 -1.575455 -0.974298  ...  -39.777626 -2.792238  -2.073951
3122  0.744666 -2.578974 -1.114246  ... -247.908573 -1.722830  -7.884967
3123  0.915846 -0.929133 -0.111333  ... -206.580638 -5.930252 -13.745464
3124  0.538809 -1.013813 -0.434048  ... -100.845205 -3.246825   1.583578

[3125 rows x 24 columns]

In [56]:
y_train

0       37
1       26
2        3
3       26
4       26
        ..
3120    26
3121    26
3122    15
3123    26
3124     2
Name: label, Length: 3125, dtype: int64

In [64]:
X_test.reset_index(inplace=True)
X_test.drop(['id'],axis=1,inplace=True)
X_test

acc_x                      ...        gy_z                      
          max       min      mean  ...         min       mean     median
0   -0.275446 -1.564000 -1.018731  ...  -35.446915  -2.000683  -1.607847
1    0.627571 -1.929033 -0.522843  ... -147.597574  -3.604579 -10.833508
2    2.972063 -0.792916  0.506947  ... -154.477074  -0.393175  -3.087735
3    0.337281 -1.045889 -0.577603  ... -117.297766  -0.024318  -0.404583
4    0.015642 -2.153047 -0.738640  ... -125.598600   5.745498   1.866352
..        ...       ...       ...  ...         ...        ...        ...
777  0.427159 -2.050254 -0.907299  ... -241.419881  -0.429678   0.082838
778  1.659451 -1.709527 -0.608731  ... -164.337764  13.272141  12.650478
779 -0.085249 -2.124959 -0.753193  ... -150.012379   1.935083   0.607579
780  1.438345  0.536568  0.958903  ...  -51.905231   0.932313  -3.805979
781  0.224194 -1.534616 -0.781752  ... -190.358982  -7.197783  -3.206301

[782 rows x 24 columns]

In [101]:
import xgboost as xgb

params = {'eta': 0.1,
          'objective': 'multi:softprob',
          'max_depth' : 3,
          'eval_metric': 'mlogloss',
          'seed' : 0,
          'lambda' : 3,
          'num_class' : 61
          }

dtrain = xgb.DMatrix(X_train, y_train)
dtest = xgb.DMatrix(X_test)

model = xgb.cv(
    params=params,
    dtrain=dtrain,
    num_boost_round=200,
    nfold=3,
    early_stopping_rounds=100,
    verbose_eval = 50
)

# Fit
final_gb = xgb.train(params, dtrain, num_boost_round=len(model))

preds = final_gb.predict(dtest)
preds

[0]	train-mlogloss:3.09451+0.00807571	test-mlogloss:3.14939+0.00736403
[50]	train-mlogloss:0.439438+0.0129337	test-mlogloss:1.26301+0.0690112
[100]	train-mlogloss:0.172241+0.00704309	test-mlogloss:1.172+0.0803786
[150]	train-mlogloss:0.0942817+0.00415502	test-mlogloss:1.16488+0.0845567
[199]	train-mlogloss:0.064243+0.00266472	test-mlogloss:1.17149+0.0846925


array([[2.59338285e-05, 4.49792678e-05, 9.00617524e-05, ...,
        4.28810623e-03, 3.09632778e-05, 6.26610417e-05],
       [9.47601919e-04, 1.31842535e-04, 4.71084379e-04, ...,
        2.99004838e-04, 1.70213534e-04, 4.54110606e-03],
       [6.93700160e-04, 2.02644005e-01, 1.79636103e-04, ...,
        2.28679171e-04, 5.86189687e-01, 2.07390357e-03],
       ...,
       [6.75601652e-04, 7.06463979e-05, 2.68411241e-04, ...,
        1.13305614e-04, 8.64919275e-05, 6.96226396e-03],
       [2.39191577e-05, 8.54820973e-05, 4.03342383e-05, ...,
        4.93322768e-05, 7.01001190e-05, 2.13918844e-04],
       [1.27118154e-04, 9.26599023e-05, 2.22057090e-04, ...,
        1.15188952e-04, 7.34986752e-05, 1.83831085e-04]], dtype=float32)

In [93]:
submission.iloc[:,1:] = preds
submission

,id,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60
0,3125,0.000730,0.000538,0.000877,0.000706,0.000921,0.000444,0.000593,0.000906,0.000546,0.005724,0.068458,0.024333,0.001142,0.740891,0.007926,0.001989,0.000341,0.000510,0.000317,0.000578,0.000800,0.000600,0.001515,0.001304,0.037928,0.000474,0.009550,0.000583,0.000490,0.000466,0.007975,0.002488,0.003592,0.000537,0.001035,0.000438,0.004867,0.000657,0.001121,0.000547,0.000533,0.000543,0.002447,0.000509,0.000575,0.000663,0.000544,0.000683,0.000514,0.001091,0.000500,0.025720,0.000757,0.001592,0.001392,0.000543,0.000489,0.001050,0.023528,0.000548,0.000347
1,3126,0.003405,0.000544,0.000657,0.001892,0.001258,0.000894,0.000594,0.002051,0.001834,0.000820,0.001053,0.000498,0.000500,0.001014,0.000660,0.001082,0.000642,0.000538,0.000318,0.000579,0.003341,0.001940,0.003417,0.044738,0.000559,0.000505,0.884101,0.000625,0.000414,0.000552,0.000903,0.000796,0.000794,0.000553,0.006431,0.002999,0.000717,0.000762,0.000858,0.000680,0.000806,0.000766,0.000936,0.001088,0.000534,0.000608,0.000538,0.000704,0.001555,0.001216,0.000909,0.001130,0.000653,0.000977,0.000707,0.000626,0.001230,0.001688,0.000903,0.000549,0.005358
2,3127,0.004155,0.186930,0.001063,0.005505,0.001405,0.002766,0.001214,0.003141,0.007871,0.002391,0.001027,0.000508,0.001021,0.001593,0.001173,0.001680,0.005749,0.006912,0.000970,0.001208,0.000885,0.002369,0.001228,0.001220,0.001206,0.001166,0.097594,0.035702,0.001213,0.001563,0.001278,0.000878,0.003842,0.022453,0.004340,0.002661,0.003490,0.002517,0.001431,0.001376,0.023862,0.006384,0.000947,0.001401,0.003991,0.011081,0.001207,0.009062,0.003270,0.004600,0.000673,0.001206,0.001291,0.001399,0.008514,0.002329,0.038955,0.001330,0.000981,0.446863,0.003961
3,3128,0.009036,0.000933,0.002172,0.000579,0.000922,0.000653,0.000781,0.000770,0.001057,0.000528,0.000348,0.000657,0.000657,0.000813,0.000607,0.001496,0.000624,0.000698,0.000418,0.000762,0.000571,0.020775,0.000606,0.000598,0.001145,0.000625,0.864992,0.000605,0.000749,0.001099,0.000563,0.000565,0.004234,0.000663,0.003877,0.001973,0.002861,0.000686,0.000732,0.000721,0.000597,0.000706,0.003200,0.003717,0.000622,0.000562,0.000697,0.000788,0.002721,0.025077,0.004648,0.000726,0.000957,0.000974,0.000668,0.000710,0.000595,0.003588,0.000608,0.000722,0.013967
4,3129,0.004773,0.000294,0.000487,0.000664,0.000742,0.000592,0.000324,0.000405,0.001582,0.000530,0.000171,0.000236,0.000273,0.000315,0.000287,0.000710,0.000257,0.000279,0.000566,0.000316,0.000902,0.000320,0.001770,0.002418,0.000157,0.000469,0.960390,0.000257,0.000285,0.000255,0.000377,0.000225,0.000614,0.000424,0.005332,0.000398,0.000334,0.000285,0.000554,0.000299,0.000313,0.000334,0.000590,0.000205,0.000258,0.000267,0.000290,0.000327,0.000416,0.001043,0.001921,0.000314,0.000367,0.000391,0.000237,0.000290,0.000300,0.001160,0.000389,0.000300,0.000420
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
777,3902,0.014590,0.000804,0.000939,0.000962,0.074957,0.031273,0.000901,0.001102,0.002122,0.004285,0.005068,0.023355,0.000806,0.004192,0.001352,0.005936,0.000708,0.000839,0.001881,0.000879,0.008468,0.001272,0.002817,0.001554,0.002477,0.001740,0.696947,0.000884,0.003343,0.000708,0.008127,0.001711,0.001626,0.000780,0.017900,0.002389,0.003445,0.000820,0.001452,0.000832,0.001793,0.002748,0.002190,0.001509,0.000844,0.000741,0.000896,0.001026,0.014502,0.012764,0.001650,0.007205,0.001020,0.002122,0.001091,0.001030,0.002034,0.001313,0.001228,0.001147,0.004905
778,3903,0.002448,0.000358,0.000601,0.000438,0.000765,0.001758,0.000302,0.000441,0.000186,0.000916,0.000239,0.000254,0.000270,0.000328,0.000282,0.002650,0.000376,0.000462,0.000537,0.000295,0.001882,0.000493,0.000546,0.001295,0.000512

In [94]:
os.chdir('/content/drive/MyDrive/데이콘/exercise_motion_classification/submission/')
submission.to_csv('xgb_model_ver1.csv', index=False)